# Zadanie domowe

W przypadku obrazów w odcieniach szarości pojedynczy piksel z zakresu [0; 255] reprezentowany jest jako 8-bitowa liczba bez znaku.
Pewnym rozszerzeniem analizy sposobu reprezentacji obrazu może być następujący eksperyment.
Załóżmy, że z każdego z 8 bitów możemy stworzyć pojedynczy obraz binarny (ang. _bit-plane slicing_).
Dla obrazka _100zloty.jpg_ (https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/100zloty.jpg) stwórz 8 obrazów, z których każdy powinien zawierać jedną płaszczyznę bitową.
Podpowiedź $-$ warto sprawdzić, jak realizuje się bitowe operacje logiczne.
Zastosowanie takiej dekompozycji obrazu pozwala na analizę ,,ważności'' poszczególnych bitów.
Jest to użyteczne w kwantyzacji, ale także w kompresji.

W drugim etapie zadania proszę spróbować odtworzyć obraz oryginalny z mniejszej liczby obrazów binarnych.
Warto zacząć od dwóch najbardziej znaczących bitów, a później dodawać kolejne.
Należy utworzyć co najmniej trzy wersje zrekonstruowanych obrazów.
Podpowiedź $-$ rekonstrukcja obrazu to mnożenie przez odpowiednią potęgę liczby 2 (przesunięcie bitowe) oraz dodawanie.

In [ ]:
import cv2
import os
import requests
import matplotlib.pyplot as plt
import numpy as np
import copy

url = 'https://raw.githubusercontent.com/vision-agh/poc_sw/master/02_Point/'
fileName = '100zloty.jpg'
if not os.path.exists(fileName) :
    r = requests.get(url + fileName, allow_redirects=True)
    open(fileName, 'wb').write(r.content)

def bit_slice(img, mask, mask_level, show_mode):
  if show_mode == 1:
    f, ax1 = plt.subplots(1,1)
    ax1.imshow(cv2.bitwise_and(img, mask))
    ax1.set_axis_off()
    ax1.set_title('slice throught bit ' + str(mask_level))
    plt.show()
  else:
    return cv2.bitwise_and(img, mask)

def bit_unslice(img0, img1, mask_level):
  result = cv2.bitwise_or(img0, img1)
  f, ax1 = plt.subplots(1,1)
  ax1.imshow(result)
  ax1.set_axis_off()
  ax1.set_title('combined bits ' + str(mask_level))
  plt.show()
  return result

money = plt.imread('100zloty.jpg')
grey_money = cv2.cvtColor(money, cv2.COLOR_BGR2GRAY)
grey_money_x, grey_money_y = grey_money.shape
plt.axis('off')
plt.gray()
plt.title('original grey scale')
plt.imshow(grey_money)

bitmask = np.ones(grey_money.shape).astype('uint8')
bit_sliced_images = np.ndarray((8, grey_money_x, grey_money_y), dtype='uint8')

'''
dla mode = 0 program zwróci wszysktie obrazy binarne
dla mode = 1 program zwróci stopniową rekonstrukcję obrazu pierwotnego
'''
mode = 0
for i in range(8):
  if mode == 0:  
    bit_slice(grey_money, bitmask, i, 1)
  else:
    bit_sliced_images[i] = bit_slice(grey_money, bitmask, i, 0)
  bitmask = bitmask * 2

if mode == 1:
  combined = copy.deepcopy(bit_sliced_images[7])
  for i in range(7,-1,-1):
    combined = bit_unslice(combined, bit_sliced_images[i], 'from 7 to ' + str(i))
